<a href="https://colab.research.google.com/github/Ehsan77e/ASR-with-Diarization/blob/main/kenlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing kenlm and compiling it

In [ ]:
!git clone https://github.com/kpu/kenlm                                                 # download version to install inside here

In [ ]:
!pip -q install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode            # install pypy interface

In [ ]:
! sudo apt -y install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

In [ ]:
! wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
! mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2       # used to compile
! ls kenlm/build/bin


# preparing dataset

In [ ]:
import pandas as pd

## Reading all datasets at once

<p>what we are doing is that we are reading all the .csv datasets from our pandas dataframes and writing all of the texts into one big text file</p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os

dir = 'path to your datasets'

for filename in os.listdir(dir):
    if filename[-4:] == ".csv":
        print(dir + '/' + filename)
        df = pd.read_csv(dir + '/' + filename)
        with open('text.txt', 'a') as f:
            for sentence in df['sentence']:
                f.write(sentence)
                f.write(' ')

/content/drive/MyDrive/asr_related/text_for_lm/filimocom.csv
/content/drive/MyDrive/asr_related/text_for_lm/digikala.csv
/content/drive/MyDrive/asr_related/text_for_lm/namava_ir.csv
/content/drive/MyDrive/asr_related/text_for_lm/Snappfood - Sentiment Analysis.csv
/content/drive/MyDrive/asr_related/text_for_lm/bbcpersian.csv
/content/drive/MyDrive/asr_related/text_for_lm/quotes.csv
/content/drive/MyDrive/asr_related/text_for_lm/taghche.csv
/content/drive/MyDrive/asr_related/text_for_lm/tweeter_anger.csv
/content/drive/MyDrive/asr_related/text_for_lm/tweeter_disgust.csv
/content/drive/MyDrive/asr_related/text_for_lm/tweeter_fear.csv
/content/drive/MyDrive/asr_related/text_for_lm/tweeter_joy.csv
/content/drive/MyDrive/asr_related/text_for_lm/tweeter_sad.csv
/content/drive/MyDrive/asr_related/text_for_lm/tweeter_surprise.csv
/content/drive/MyDrive/asr_related/text_for_lm/virgool_dataset.csv
/content/drive/MyDrive/asr_related/text_for_lm/youtube1.csv
/content/drive/MyDrive/asr_related/text_

# creating the LM

In [ ]:
! kenlm/build/bin/lmplz -o 5 < "text.txt" > "5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 72061755 types 693501
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:8322012 2:1061687040 3:1990663296 4:3185061120 5:4644881408
Statistics:
1 693500 D1=0.737434 D2=0.991693 D3+=1.24765
2 12577171 D1=0.769346 D2=1.0595 D3+=1.32052
3 40029963 D1=0.870311 D2=1.14869 D3+=1.32117
4 57730708 D1=0.944196 D2=1.25759 D3+=1.38309
5 64338627 D1=0.942289 D2=1.49521 D3+=1.58849
Memory estimate for binary LM:
type      MB
probing 3647 assuming -p 1.5
probing 4281 assuming -r models -p 1.5
trie    1840 without quantization
trie    1046 assuming -q 8 -b 8 quantization 
trie    1596 assuming -a 22 array pointer compression
trie     801 assuming -a 22 -q 8 -b 8 array pointer compression and quantization


In [ ]:
!head -20 5gram.arpa

\data\
ngram 1=693500
ngram 2=12577171
ngram 3=40029963
ngram 4=57730708
ngram 5=64338627

\1-grams:
-7.1485944	<unk>	0
0	<s>	-0.11387854
-3.140654	خدا	-0.7478481
-5.5355606	شاهده	-0.14423017
-3.9437475	مثله	-0.49795055
-3.03724	چی	-0.8094707
-3.8682036	نت	-0.593545
-3.7188933	میخوره	-0.5836112
-3.8443959	ببینید	-0.53187865
-3.24348	مردم	-0.7148126
-4.6101227	چیا	-0.29718775
-4.0731945	نوشتن	-0.44905248


# adding the s and /s tokens for each sentence

In [ ]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)


In [ ]:
!head -20 5gram_correct.arpa

\data\
ngram 1=693501
ngram 2=12577171
ngram 3=40029963
ngram 4=57730708
ngram 5=64338627

\1-grams:
-7.1485944	<unk>	0
0	<s>	-0.11387854
0	</s>	-0.11387854
-3.140654	خدا	-0.7478481
-5.5355606	شاهده	-0.14423017
-3.9437475	مثله	-0.49795055
-3.03724	چی	-0.8094707
-3.8682036	نت	-0.593545
-3.7188933	میخوره	-0.5836112
-3.8443959	ببینید	-0.53187865
-3.24348	مردم	-0.7148126
-4.6101227	چیا	-0.29718775


In [ ]:
# import kenlm
# model = kenlm.LanguageModel('5gram_correct.arpa')

In [ ]:
# print(model.score('سلام بر دوستان عزیز', bos=True, eos=True))

-9.11422061920166


# combining it with wav2vec2

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00


In [ ]:
model_name_or_path = "replace it with your own model name or the path to it"

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(model_name_or_path)
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [ ]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

# saving the lm based decoder

In [ ]:
# processor_with_lm.save_pretrained(model_name_or_path)
lm_name_or_path = '/content/drive/MyDrive/asr_related/lm-decoder'  # replace it with your own desired path
processor_with_lm.save_pretrained(lm_name_or_path)



In [ ]:
!sudo apt-get install git-lfs tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (172 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading d

# reducing the size of lm by turning it to .bin format

In [2]:
!kenlm/build/bin/build_binary $lm_name_or_path/language_model/5gram_correct.arpa $lm_name_or_path/language_model/5gram.bin


/bin/bash: line 1: kenlm/build/bin/build_binary: No such file or directory


# with the bin file now you can delete .arpa file

In [ ]:
!rm $lm_name_or_path/language_model/5gram_correct.arpa

In [ ]:
!tree -h $lm_name_or_path

[4.0K]  /content/drive/MyDrive/asr_related/lm-decoder
├── [ 413]  alphabet.json
├── [4.0K]  language_model
│   ├── [3.6G]  5gram.bin
│   ├── [  78]  attrs.json
│   └── [ 10M]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [  96]  special_tokens_map.json
├── [1.1K]  tokenizer_config.json
└── [ 482]  vocab.json

1 directory, 8 files


# the lm bades decoder is now created and you can you use it now

# tying it out with the ASR system

In [ ]:
import transformers

In [ ]:
asr_model = transformers.Wav2Vec2ForCTC.from_pretrained('model_name_or_path')

Some weights of the model checkpoint at /content/drive/MyDrive/asr_related/model_training were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /content/drive/MyDrive/asr_related/model_training and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.origi

In [ ]:
audio_path = 'sample_audio_path'

In [ ]:
import IPython.display as display

display.Audio(audio_path)

In [ ]:
import librosa
speech, rate = librosa.load(audio_path, sr=16000)

In [ ]:
input_values = processor_with_lm(speech, sampling_rate=16_000, return_tensors='pt')#.to(torch.device("cuda"))


In [ ]:
import torch

In [ ]:
with torch.no_grad():
    logits = asr_model(input_values.input_values, attention_mask=input_values.attention_mask).logits

In [ ]:
#logits = asr_model(input_values).logits
#logits = logits.to('cpu')  # Copy the tensor to CPU memory
logits_np = logits.numpy()  # Convert the tensor to a NumPy array


In [ ]:
transcription = processor_with_lm.batch_decode(logits_np).text
print(transcription)

['جون الو سلام خوبی سلام مرسی سلام بین یه سوال این درخواست نبود گفتم که میخوام انتقال اعتبار برای دانشجو ها غیر فعال شه خانوم مستوی داده بوس برنامه نویس خیلی وقت بود مونده بود خوب یعنی از اوایل فیبما اون روز ده پیگیری می کنم برنامه نویس انجام بده الان دیدم کشمو بستن ولی همچنان فعاله اون قسمت علتش چیه گوشی داشته باشه مر']
